# 03_modeling

Panel regressions linking **poverty**, **median household income**, and **unemployment** using a state-year panel.

**Inputs:** `data/processed/state_year_panel.parquet` (built in `01_build_state_year_panel.ipynb`)

In [1]:
from pathlib import Path
import pandas as pd

# =====================================================
# Block 1: Locate repo + load processed panel
# =====================================================

START = Path.cwd().resolve()

def find_repo(start: Path) -> Path:
    """
    Find repo root by searching for data/raw containing required inputs.
    Works even when the Jupyter kernel starts outside the repo (e.g., C:\\projects).
    """
    roots = [start]
    if start.drive:  # Windows: also try common dev root
        roots.append(Path(start.drive + "\\") / "projects")

    required_raw_files = [
        "saipe_state_year.parquet",
        "laus_allstates_u.txt",
        "la.area.txt",
        "la.series.txt",
    ]

    seen = set()
    for root in roots:
        root = root.resolve()
        if root in seen or not root.exists():
            continue
        seen.add(root)

        for raw_dir in root.rglob("data/raw"):
            if all((raw_dir / f).exists() for f in required_raw_files):
                return raw_dir.parent.parent  # .../data/raw -> repo root

    raise RuntimeError(
        f"Could not find repo root from start={start}. "
        f"Expected to find data/raw containing: {required_raw_files}"
    )

REPO = find_repo(START)
PROCESSED = REPO / "data" / "processed"
PANEL_PATH = PROCESSED / "state_year_panel.parquet"

print("Auto-found REPO:", REPO)
print("Looking for:", PANEL_PATH)
print("Processed files:", [p.name for p in PROCESSED.glob("*")])

assert PANEL_PATH.exists(), f"Missing processed panel: {PANEL_PATH}"

panel = pd.read_parquet(PANEL_PATH)
print("Loaded panel:", panel.shape)
display(panel.head())


Auto-found REPO: C:\projects\python-policy-project
Looking for: C:\projects\python-policy-project\data\processed\state_year_panel.parquet
Processed files: ['.gitkeep', 'laus_state_year_unemployment_rate.parquet', 'saipe_state_year_clean.parquet', 'state_year_panel.csv', 'state_year_panel.parquet', 'state_year_panel_model_ready.parquet']
Loaded panel: (1581, 7)


,state,state_name,state_fips,year,poverty_rate,median_income,unemployment_rate
0,AK,Alaska,02,1989,10.6,33885,7.1
1,AK,Alaska,02,1993,11.2,39431,7.8
2,AK,Alaska,02,1995,10.1,42255,7.3
3,AK,Alaska,02,1996,10.6,44797,7.5
4,AK,Alaska,02,1997,11.2,43657,7.0


In [2]:
import numpy as np

# =====================================================
# Block 2: Define modeling sample
# =====================================================

df = panel.loc[
    panel["year"] >= 1995,
    ["state", "year", "poverty_rate", "median_income", "unemployment_rate"]
].dropna().copy()

# Log income for percent-change interpretation
df["log_income"] = np.log(df["median_income"].astype(float))

# Patsy/statsmodels compatibility: avoid pandas nullable Int64 dtype
df["year"] = df["year"].astype(int)

print("Modeling df shape:", df.shape)
print("Years:", int(df['year'].min()), "-", int(df['year'].max()))
print("States per year (tail):")
print(df.groupby("year")["state"].nunique().tail())

display(df.head())


Modeling df shape: (1479, 6)
Years: 1995 - 2023
States per year (tail):
year
2019    51
2020    51
2021    51
2022    51
2023    51
Name: state, dtype: int64


,state,year,poverty_rate,median_income,unemployment_rate,log_income
2,AK,1995,10.1,42255,7.3,10.651478
3,AK,1996,10.6,44797,7.5,10.709896
4,AK,1997,11.2,43657,7.0,10.684119
5,AK,1998,10.8,47177,6.3,10.761662
6,AK,1999,8.8,49133,6.4,10.802286


In [3]:
# =====================================================
# Block 3: Modeling backend
# =====================================================

import statsmodels.formula.api as smf


In [4]:
# =====================================================
# Block 4: Pooled OLS models (intuition)
# =====================================================

# Poverty ~ Unemployment
m1 = smf.ols("poverty_rate ~ unemployment_rate", data=df).fit(cov_type="HC1")

# Poverty ~ Income (levels)
m2 = smf.ols("poverty_rate ~ median_income", data=df).fit(cov_type="HC1")

# Poverty ~ Unemployment + Income (levels)
m3 = smf.ols("poverty_rate ~ unemployment_rate + median_income", data=df).fit(cov_type="HC1")

print(m1.summary())
print(m2.summary())
print(m3.summary())


                            OLS Regression Results                            
Dep. Variable:           poverty_rate   R-squared:                       0.200
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     326.1
Date:                Sun, 04 Jan 2026   Prob (F-statistic):           5.13e-66
Time:                        01:30:33   Log-Likelihood:                -3663.6
No. Observations:                1479   AIC:                             7331.
Df Residuals:                    1477   BIC:                             7342.
Df Model:                           1                                         
Covariance Type:                  HC1                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             9.0722      0.21

In [5]:
# =====================================================
# Block 5: Pooled OLS with log income (cleaner interpretation)
# =====================================================

m4 = smf.ols("poverty_rate ~ unemployment_rate + log_income", data=df).fit(cov_type="HC1")
print(m4.summary())

# --- NOTE ---
# Interpreting log_income:
# A 10% increase in income is approximately associated with:
#   0.10 * coef(log_income) percentage-point change in poverty_rate.


                            OLS Regression Results                            
Dep. Variable:           poverty_rate   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.317
Method:                 Least Squares   F-statistic:                     416.9
Date:                Sun, 04 Jan 2026   Prob (F-statistic):          2.91e-144
Time:                        01:30:33   Log-Likelihood:                -3545.9
No. Observations:                1479   AIC:                             7098.
Df Residuals:                    1476   BIC:                             7114.
Df Model:                           2                                         
Covariance Type:                  HC1                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            54.2528      2.67

In [6]:
# =====================================================
# Block 6: Two-way fixed effects (state + year), clustered SEs
# =====================================================

m_twfe = smf.ols(
    "poverty_rate ~ unemployment_rate + log_income + C(state) + C(year)",
    data=df
).fit(
    cov_type="cluster",
    cov_kwds={"groups": df["state"]}
)

print(m_twfe.summary())


                            OLS Regression Results                            
Dep. Variable:           poverty_rate   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                     796.3
Date:                Sun, 04 Jan 2026   Prob (F-statistic):           1.46e-57
Time:                        01:30:33   Log-Likelihood:                -1130.9
No. Observations:                1479   AIC:                             2424.
Df Residuals:                    1398   BIC:                             2853.
Df Model:                          80                                         
Covariance Type:              cluster                                         
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           146.8278     12.79

c:\projects\python-policy-project\.venv\Lib\site-packages\statsmodels\base\model.py:1894: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 80, but rank is 30
  warnings.warn('covariance of constraints does not have full '


In [7]:
# =====================================================
# Block 7: Key coefficients (README-ready)
# =====================================================

coef_table = (
    m_twfe.params.loc[["unemployment_rate", "log_income"]].to_frame("coef")
    .join(m_twfe.bse.loc[["unemployment_rate", "log_income"]].rename("se"))
    .join(m_twfe.pvalues.loc[["unemployment_rate", "log_income"]].rename("pvalue"))
)

display(coef_table)


,coef,se,pvalue
unemployment_rate,0.293379,0.030162,2.318493e-22
log_income,-13.006020,1.192571,1.080608e-27


In [8]:
# =====================================================
# Block 8: Simple model comparison table
# =====================================================

models = {
    "OLS: Unemp only": m1,
    "OLS: Income only": m2,
    "OLS: Both (levels)": m3,
    "OLS: Log income": m4,
    "TWFE (state+year, clustered)": m_twfe,
}

summary = pd.DataFrame({
    name: {
        "R²": round(mod.rsquared, 3),
        "Adj. R²": round(mod.rsquared_adj, 3),
        "N": int(mod.nobs),
    }
    for name, mod in models.items()
})

display(summary.T)


,R²,Adj. R²,N
OLS: Unemp only,0.200,0.200,1479.0
OLS: Income only,0.141,0.140,1479.0
OLS: Both (levels),0.307,0.306,1479.0
OLS: Log income,0.318,0.317,1479.0
"TWFE (state+year, clustered)",0.974,0.972,1479.0


## Notes / Interpretation

Pooled OLS conflates cross-state structural differences with within-state changes over time; fixed effects models isolate within-state dynamics by differencing out time-invariant state characteristics.

Median household income is strongly and robustly negatively associated with poverty. Unemployment is positively associated with poverty, but its magnitude is substantially reduced once income is controlled for.

State fixed effects absorb persistent structural differences across states (e.g., institutions, demographics, baseline policy environments), while year fixed effects capture national shocks such as recessions and COVID-era transfer programs.

Preferred specification: **state + year fixed effects with state-clustered standard errors**.


In [9]:
# =====================================================
# Block 9: Save model-ready dataset
# =====================================================

MODEL_PATH = PROCESSED / "state_year_panel_model_ready.parquet"
df.to_parquet(MODEL_PATH, index=False)
print("Saved:", MODEL_PATH)


Saved: C:\projects\python-policy-project\data\processed\state_year_panel_model_ready.parquet
